<a href="https://colab.research.google.com/github/sheikh495/HierarchySimulator/blob/main/Hierarchy_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here's an outline of the program that simulates the behavior of a memory hierarchy based on the provided requirements. The outline covers the main steps involved in the simulation:

1. Read the configuration file (trace.config) to determine the characteristics of the memory hierarchy.
2. Initialize the memory hierarchy components (DTLB, PT, DC, and L2) based on the configuration.
3. Read the trace of references from standard input, line by line.
4. For each reference, extract the access type and address.
5. If virtual address translation is enabled:
   - Perform virtual-to-physical address translation using the DTLB and PT.
   - Update the TLB, PT, and associated caches accordingly.
6. If the DC is enabled:
   - Simulate the behavior of the DC by checking for hits and misses.
   - Update the DC cache based on the access type.
7. If the L2 cache is enabled:
   - Simulate the behavior of the L2 cache by checking for hits and misses.
   - Update the L2 cache based on the access type.
8. Keep track of the hit and miss counts for each level of the hierarchy.
9. Calculate and print the configuration of the memory hierarchy.
10. For each reference, print the relevant information (virtual address, virtual page number, page offset, TLB tag, TLB index, TLB result, PT result, physical page number, DC tag, DC index, DC result, L2 tag, L2 index, L2 result).
11. Print the number of hits, misses, and hit ratio for each level of the hierarchy.
12. Print the number of reads, writes, read ratio, total memory references, accesses to the page table, and disk references.

Please note that the above outline provides a high-level overview of the steps involved in simulating the memory hierarchy. Implementing the detailed logic and data structures for each component (DTLB, PT, DC, and L2) will require additional code and appropriate algorithms.



In [ ]:
class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components (DTLB, PT, DC, L2) based on the configuration

        # Initialize the DTLB
        if self.config_data.get('TLB', 'n') == 'y':
            tlb_num_sets = int(self.config_data.get('Number of sets TLB', '1'))
            tlb_set_size = int(self.config_data.get('Set size TLB', '1'))
            self.dtlb = TLBCache(tlb_num_sets, tlb_set_size)

        # Initialize the PT
        page_size = int(self.config_data.get('Page size', '4096'))
        virtual_pages = int(self.config_data.get('Number of virtual pages', '8192'))
        physical_pages = int(self.config_data.get('Number of physical pages', '1024'))
        self.pt = PageTable(page_size, virtual_pages, physical_pages)

        # Initialize the DC
        if self.config_data.get('DC', 'n') == 'y':
            dc_num_sets = int(self.config_data.get('Number of sets DC', '1'))
            dc_set_size = int(self.config_data.get('Set size DC', '1'))
            dc_line_size = int(self.config_data.get('Line size DC', '16'))
            write_through = self.config_data.get('Write through/no write allocate', 'n') == 'y'
            self.dc = DataCache(dc_num_sets, dc_set_size, dc_line_size, write_through)

        # Initialize the L2 cache
        if self.config_data.get('L2 cache', 'n') == 'y':
            l2_num_sets = int(self.config_data.get('Number of sets L2', '1'))
            l2_set_size = int(self.config_data.get('Set size L2', '1'))
            l2_line_size = int(self.config_data.get('Line size L2', '16'))
            write_through = self.config_data.get('Write through/no write allocate', 'n') == 'y'
            self.l2 = DataCache(l2_num_sets, l2_set_size, l2_line_size, write_through)

 # class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

    # ... (existing code)


    def print_configuration(self):
         print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")


    def print_reference_information(self):
        print("Information about each Reference")
        print("Virtual Virt. Page TLB TLB TLB PT Phys DC DC L2 L2")
        print("Address Page # Off Tag Ind Res. Res. Pg # DC Tag Ind Res. L2 Tag Ind Res.")
        print("-------- ------ ---- ------ --- ---- ---- ---- ------ --- ---- ------ --- ----")

        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Obtain the relevant information
                virtual_page_number = virtual_address >> self.pt.page_offset_bits
                page_offset = virtual_address & ((1 << self.pt.page_offset_bits) - 1)
                tlb_tag, tlb_index, tlb_result = self.dtlb.lookup_info(virtual_page_number)
                pt_result = self.pt.lookup_info(virtual_page_number)
                physical_page_number = pt_result['physical_page_number'] if pt_result else None
                dc_tag, dc_index, dc_result = self.dc.lookup_info(physical_page_number) if physical_page_number else (None, None, None)
                l2_tag, l2_index, l2_result = self.l2.lookup_info(physical_page_number) if physical_page_number else (None, None, None)

                # Print the relevant information
                print(f"{address} {virtual_page_number:6x} {page_offset:4x} {tlb_tag:6x} {tlb_index:3x} {tlb_result:4s} {pt_result['result']:4s} {physical_page_number:4x} {dc_tag:6x} {dc_index:3x} {dc_result:4s} {l2_tag:6x} {l2_index:3x} {l2_result:4s}")


    def print_simulation_statistics(self):
             print("Simulation Statistics")

        # Print the number of hits, misses, and hit ratio for each level of the hierarchy
        print("DTLB:")
        print(f"DTLB hits: {self.hits['dtlb']}")
        print(f"DTLB misses: {self.misses['dtlb']}")
        dtlb_hit_ratio = self.hits['dtlb'] / (self.hits['dtlb'] + self.misses['dtlb'])
        print(f"DTLB hit ratio: {dtlb_hit_ratio:.6f}")

        print("Page Table:")
        print(f"PT hits: {self.hits['pt']}")
        print(f"PT faults: {self.misses['pt']}")
        pt_hit_ratio = self.hits['pt'] / (self.hits['pt'] + self.misses['pt'])
        print(f"PT hit ratio: {pt_hit_ratio:.6f}")

        if self.config_data.get('DC', 'n') == 'y':
            print("Data Cache:")
            print(f"DC hits: {self.hits['dc']}")
            print(f"DC misses: {self.misses['dc']}")
            dc_hit_ratio = self.hits['dc'] / (self.hits['dc'] + self.misses['dc'])
            print(f"DC hit ratio: {dc_hit_ratio:.6f}")

        if self.config_data.get('L2 cache', 'n') == 'y':
            print("L2 Cache:")
            print(f"L2 hits: {self.hits['l2']}")
            print(f"L2 misses: {self.misses['l2']}")
            l2_hit_ratio = self.hits['l2'] / (self.hits['l2'] + self.misses['l2'])
            print(f"L2 hit ratio: {l2_hit_ratio:.6f}")

        # Print the number of reads, writes, read ratio, total memory references, accesses to the page table, and disk references
        total_reads = self.accesses['R']
        total_writes = self.accesses['W']
        total_memory_references = total_reads + total_writes
        read_ratio = total_reads / total_memory_references

        print(f"Total reads: {total_reads}")
        print(f"Total writes: {total_writes}")
        print(f"Read ratio: {read_ratio:.6f}")
        print(f"Total memory references: {total_memory_references}")
        print(f"Accesses to the page table: {self.page_table_accesses}")
        print(f"Disk references: {self.disk_references}")


# Usage Example:
simulator = MemoryHierarchySimulator('E:/UTK/Summer/Couputer systems Organization CS-530/trace.config', 'E:/UTK/Summer/Couputer systems Organization CS-530/trace.dat')
simulator.read_configuration()
simulator.initialize_memory_hierarchy()
simulator.read_trace_data()
simulator.simulate_memory_hierarchy()
simulator.print_configuration()
simulator.print_reference_information()
simulator.print_simulation_statistics()



IndentationError: ignored

In [ ]:
# Upload trace.config and trace.dat files
uploaded = files.upload()

# Get the file paths
config_file_path = next(iter(uploaded))
trace_file_path = next(iter(uploaded))

# Create the simulator instance
simulator = MemoryHierarchySimulator(config_file_path, trace_file_path)

# Read the configuration and trace data
simulator.read_configuration()
simulator.initialize_memory_hierarchy()
simulator.read_trace_data()

# Simulate the memory hierarchy
simulator.simulate_memory_hierarchy()

# Print the configuration, reference information, and simulation statistics
simulator.print_configuration()
simulator.print_reference_information()
simulator.print_simulation_statistics()


NameError: ignored

1

In [29]:
from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

    # ... (existing code)


    def print_configuration(self):
    #def print_configuration(self):
        print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")

2

In [31]:
class DTLB:
    def __init__(self, num_sets, set_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.cache = [[None] * set_size for _ in range(num_sets)]

    def lookup(self, virtual_address):
        set_index = (virtual_address >> 6) % self.num_sets
        tag = virtual_address >> 12

        for entry in self.cache[set_index]:
            if entry is not None and entry['tag'] == tag:
                return entry

        return None

    def update(self, virtual_address, physical_address):
        set_index = (virtual_address >> 6) % self.num_sets
        tag = virtual_address >> 12

        for i, entry in enumerate(self.cache[set_index]):
            if entry is None or entry['tag'] == tag:
                self.cache[set_index][i] = {'tag': tag, 'physical_address': physical_address}
                return

        # If all entries are occupied, replace the oldest entry
        self.cache[set_index][0] = {'tag': tag, 'physical_address': physical_address}


In [32]:


from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                 #    config_file = files.upload()
                      config_file_path = list(config_file.keys())[0]
                      self.config_file_path = config_file_path

                 # Upload trace.dat file
    trace_file = files.upload()
    trace_file_path = list(trace_file.keys())[0]
    self.trace_file_path = trace_file_path

    # Read the configuration from the uploaded trace.config file
    self.read_configuration()

    # Initialize the memory hierarchy components
    self.initialize_memory_hierarchy()

    # Read the trace data from the uploaded trace.dat file
    self.read_trace_data()
                  #  self.uploaded = files.upload()
                 # lTo use the code in Google Colab, you need to upload the `trace.config` and `trace.dat` files to the Colab runtime environment. You can do this using the file upload feature in Colab. Here's an updated version of the code that includes the file upload steps:



Saving trace.config to trace (5).config


NameError: ignored

3

In [ ]:

from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

    # ... (existing code)


    def print_configuration(self):
        print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")


    def print_reference_information(self):
          #def print_simulation_statistics(self):
        print("Simulation Statistics")

        # Print the number of hits, misses, and hit ratio for each level of the hierarchy
        print("DTLB:")
        print(f"DTLB hits: {self.hits['dtlb']}")
        print(f"DTLB misses: {self.misses['dtlb']}")
        dtlb_hit_ratio = self.hits['dtlb'] / (self.hits['dtlb'] + self.misses['dtlb'])
        print(f"DTLB hit ratio: {dtlb_hit_ratio:.6f}")

        print("Page Table:")
        print(f"PT hits: {self.hits['pt']}")
        print(f"PT faults: {self.misses['pt']}")
        pt_hit_ratio = self.hits['pt'] / (self.hits['pt'] + self.misses['pt'])
        print(f"PT hit ratio: {pt_hit_ratio:.6f}")

        if self.config_data.get('DC', 'n') == 'y':
            print("Data Cache:")
            print(f"DC hits: {self.hits['dc']}")
            print(f"DC misses: {self.misses['dc']}")
            dc_hit_ratio = self.hits['dc'] / (self.hits['dc'] + self.misses['dc'])
            print(f"DC hit ratio: {dc_hit_ratio:.6f}")

        if self.config_data.get('L2 cache', 'n') == 'y':
            print("L2 Cache:")
            print(f"L2 hits: {self.hits['l2']}")
            print(f"L2 misses: {self.misses['l2']}")
            l2_hit_ratio = self.hits['l2'] / (self.hits['l2'] + self.misses['l2'])
            print(f"L2 hit ratio: {l2_hit_ratio:.6f}")

        # Print the number of reads, writes, read ratio, total memory references, accesses to the page table, and disk references
        total_reads = self.accesses['R']
        total_writes = self.accesses['W']
        total_memory_references = total_reads + total_writes
        read_ratio = total_reads / total_memory_references

        print(f"Total reads: {total_reads}")
        print(f"Total writes: {total_writes}")
        print(f"Read ratio: {read_ratio:.6f}")
        print(f"Total memory references: {total_memory_references}")
        print(f"Accesses to the page table: {self.page_table_accesses}")
        print(f"Disk references: {self.disk_references}")


4

In [87]:
from google.colab import files
class DTLB:
    def __init__(self, num_sets, set_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.cache = [[None] * set_size for _ in range(num_sets)]

    def lookup(self, virtual_address):
        set_index = (virtual_address >> 6) % self.num_sets
        tag = virtual_address >> 12

        for entry in self.cache[set_index]:
            if entry is not None and entry['tag'] == tag:
                return entry

        return None

    def update(self, virtual_address, physical_address):
        set_index = (virtual_address >> 6) % self.num_sets
        tag = virtual_address >> 12

        for i, entry in enumerate(self.cache[set_index]):
            if entry is None or entry['tag'] == tag:
                self.cache[set_index][i] = {'tag': tag, 'physical_address': physical_address}
                return

        # If all entries are occupied, replace the oldest entry
        self.cache[set_index][0] = {'tag': tag, 'physical_address': physical_address}
class PageTable:
    def __init__(self, num_entries):
        self.num_entries = num_entries
        self.page_table = [None] * num_entries

    def lookup(self, virtual_address):
        page_number = virtual_address >> 12

        if 0 <= page_number < self.num_entries:
            return self.page_table[page_number]

        return None

    def update(self, virtual_address, physical_address):
        page_number = virtual_address >> 12

        if 0 <= page_number < self.num_entries:
            self.page_table[page_number] = {'physical_address': physical_address}
#---
# l2 cache
class L2Cache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def access(self, address):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return True  # Hit
        else:
            cache_set[line_offset] = address
            return False  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset

#---
from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())

    # ... (existing code)

simulator = MemoryHierarchySimulator(config_file_path, trace_file_path)

# Read the configuration from the uploaded trace.config file
simulator.read_configuration()

# Initialize the memory hierarchy components
simulator.initialize_memory_hierarchy()

# Read the trace data from the uploaded trace.dat file
simulator.read_trace_data()

# ... (existing code)
#-----------
class DTLB:
    def __init__(self, num_sets, set_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def lookup(self, address):
        set_index, tag = self.get_indices(address)
        cache_set = self.cache[set_index]
        if tag in cache_set:
            return {'physical_address': address}
        else:
            return None

    def update(self, address, physical_address):
        set_index, tag = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_set.remove(None)
        cache_set.append(tag)

    def get_indices(self, address):
        set_index = (address // self.set_size) % self.num_sets
        tag = address // self.set_size
        return set_index, tag


class PageTable:
    def __init__(self, num_entries):
        self.num_entries = num_entries
        self.entries = {}

    def lookup(self, address):
        if address in self.entries:
            return {'physical_address': self.entries[address]}
        else:
            return None

    def update(self, address, physical_address):
        self.entries[address] = physical_address


class DataCache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def update_cache(self, address, is_write):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return  # Hit
        else:
            cache_set[line_offset] = address
            return  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset


class L2Cache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def update_cache(self, address, is_write):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return  # Hit
        else:
            cache_set[line_offset] = address
            return  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset


class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)

    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            if ':' in line:
                address, access_type = line.strip().split(':')
                self.trace_data.append((int(address.strip(), 16), access_type.strip()))

    def simulate(self):
        for address, access_type in self.trace_data:
            # Check DTLB
            dtlb_lookup_result = self.dtlb.lookup(address)
            if dtlb_lookup_result is None:
                self.misses['dtlb'] += 1
                dtlb_lookup_result = self.pt.lookup(address)
                if dtlb_lookup_result is None:
                    self.misses['pt'] += 1
                    physical_address = address
                else:
                    self.hits['pt'] += 1
                    physical_address = dtlb_lookup_result['physical_address']
                    self.dtlb.update(address, physical_address)
            else:
                self.hits['dtlb'] += 1
                physical_address = dtlb_lookup_result['physical_address']

            # Access DC
            self.dc.update_cache(physical_address, access_type == 'W')

            # Access L2 cache
            self.l2.update_cache(physical_address, access_type == 'W')

        print('DTLB hits:', self.hits['dtlb'])
        print('DTLB misses:', self.misses['dtlb'])
        print('PT hits:', self.hits['pt'])
        print('PT misses:', self.misses['pt'])
        print('DC hits:', self.hits['dc'])
        print('DC misses:', self.misses['dc'])
        print('L2 cache hits:', self.hits['l2'])
        print('L2 cache misses:', self.misses['l2'])

# __@@@@@@@@@@@@@@@@@@@@@@


class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()
# @@@@@@@@@@@@@@@@
class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)

    # Rest of the class implementation...
    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())
    ##############{}{}{}}{

  # Rest of the class implementation...

#class MemoryHierarchySimulator:
    # Rest of the class implementation...

    def simulate(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

        print('DTLB hits:', self.hits['dtlb'])
        print('DTLB misses:', self.misses['dtlb'])
        print('PT hits:', self.hits['pt'])
        print('PT misses:', self.misses['pt'])
        print('DC hits:', self.hits['dc'])
        print('DC misses:', self.misses['dc'])
        print('L2 cache hits:', self.hits['l2'])
        print('L2 cache misses:', self.misses['l2'])



# @@@@@@@@@@@
class DataCache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None] * set_size for _ in range(num_sets)]

    def lookup(self, address):
        set_index = (address >> 5) % self.num_sets

        for entry in self.cache[set_index]:
            if entry is not None and entry['tag'] == (address >> 5):
                return entry

        return None

    def update(self, address, is_write):
        set_index = (address >> 5) % self.num_sets
        tag = address >> 5

        for i, entry in enumerate(self.cache[set_index]):
            if entry is None:
                self.cache[set_index][i] = {'tag': tag}
                return

        # If there is no empty slot, replace the first entry
        self.cache[set_index][0] = {'tag': tag}

    def print_cache(self):
        for set_index in range(self.num_sets):
            print(f"Set {set_index}:")
            for i, entry in enumerate(self.cache[set_index]):
                if entry is not None:
                    print(f"  Entry {i}: Tag={entry['tag']}")
                else:
                    print(f"  Entry {i}: Empty")


    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    # Existing code...


    def print_configuration(self):
        print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")
        #def print_configuration(self):
        print("Configuration of the Memory Hierarchy")


    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

    # ... (existing code)


    def print_configuration(self):
        print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")


    def print_reference_information(self):
        print("Information about each Reference")
        print("Virtual Virt. Page TLB TLB TLB PT Phys DC DC L2 L2")
        print("Address Page # Off Tag Ind Res. Res. Pg # DC Tag Ind Res. L2 Tag Ind Res.")
        print("-------- ------ ---- ------ --- ---- ---- ---- ------ --- ---- ------ --- ----")

        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Obtain the relevant information
                virtual_page_number = virtual_address >> self.pt.page_offset_bits
                page_offset = virtual_address & ((1 << self.pt.page_offset_bits) - 1)
                tlb_tag, tlb_index, tlb_result = self.dtlb.lookup_info(virtual_page_number)
                pt_result = self.pt.lookup_info(virtual_page_number)

                # Print the information
                print(f"{address} {virtual_page_number} {page_offset} {tlb_tag} {tlb_index} {tlb_result} {pt_result['physical_page_number']} {pt_result['dc_tag']} {pt_result['dc_index']} {pt_result['dc_result']} {pt_result['l2_tag']} {pt_result['l2_index']} {pt_result['l2_result']}")

# Upload the configuration file
config_file = files.upload()
config_file_path = list(config_file.keys())[0]

# Upload the trace file
trace_file = files.upload()
trace_file_path = list(trace_file.keys())[0]

# Create an instance of the MemoryHierarchySimulator
simulator = MemoryHierarchySimulator(config_file_path, trace_file_path)

# Read the configuration
simulator.read_configuration()

# Initialize the memory hierarchy
simulator.initialize_memory_hierarchy()

# Read the trace data
simulator.read_trace_data()

# Simulate the memory hierarchy
#simulator.simulate_memory_hierarchy()
simulator.simulate()
# Print the configuration
#simulator.print_configuration()
def read_configuration(self):
    with open(self.config_file_path, 'r') as config_file:
        config_lines = config_file.readlines()

    for line in config_lines:
        if ':' in line:
            key, value = line.strip().split(':')
            self.config_data[key.strip()] = value.strip()
            print(f"{key.strip()}: {value.strip()}")  # Print the configuration line

# Print the reference information
#simulator.print_reference_information()
def simulate(self):
    for address, access_type in self.trace_data:
        # Check DTLB
        dtlb_lookup_result = self.dtlb.lookup(address)
        if dtlb_lookup_result is None:
            self.misses['dtlb'] += 1
            dtlb_lookup_result = self.pt.lookup(address)
            if dtlb_lookup_result is None:
                self.misses['pt'] += 1
                physical_address = address
            else:
                self.hits['pt'] += 1
                physical_address = dtlb_lookup_result['physical_address']
                self.dtlb.update(address, physical_address)
        else:
            self.hits['dtlb'] += 1
            physical_address = dtlb_lookup_result['physical_address']

        # Access DC
        self.dc.update_cache(physical_address, access_type == 'W')

        # Access L2 cache
        self.l2.update_cache(physical_address, access_type == 'W')

        # Print reference information
        print(f"Address: {address}, Access Type: {access_type}, Physical Address: {physical_address}")

    print('DTLB hits:', self.hits['dtlb'])
    print('DTLB misses:', self.misses['dtlb'])
    print('PT hits:', self.hits['pt'])
    print('PT misses:', self.misses['pt'])
    print('DC hits:', self.hits['dc'])
    print('DC misses:', self.misses['dc'])
    print('L2 cache hits:', self.hits['l2'])
    print('L2 cache misses:', self.misses['l2'])



UnicodeDecodeError: ignored

@@@@

In [ ]:

from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

    # ... (existing code)


    def print_configuration(self):
        print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")


    def print_reference_information(self):
        print("Information about each Reference")
        print("Virtual Virt. Page TLB TLB TLB PT Phys DC DC L2 L2")
        print("Address Page # Off Tag Ind Res. Res. Pg # DC Tag Ind Res. L2 Tag Ind Res.")
        print("-------- ------ ---- ------ --- ---- ---- ---- ------ --- ---- ------ --- ----")

        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Obtain the relevant information
                virtual_page_number = virtual_address >> self.pt.page_offset_bits
                page_offset = virtual_address & ((1 << self.pt.page_offset_bits) - 1)
                tlb_tag, tlb_index, tlb_result = self.dtlb.lookup_info(virtual_page_number)
                pt_result = self.pt.lookup_info(virtual_page_number)


In [49]:
class DTLB:
    def __init__(self, num_sets, set_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def lookup(self, address):
        set_index, tag = self.get_indices(address)
        cache_set = self.cache[set_index]
        if tag in cache_set:
            return {'physical_address': address}
        else:
            return None

    def update(self, address, physical_address):
        set_index, tag = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_set.remove(None)
        cache_set.append(tag)

    def get_indices(self, address):
        set_index = (address // self.set_size) % self.num_sets
        tag = address // self.set_size
        return set_index, tag


class PageTable:
    def __init__(self, num_entries):
        self.num_entries = num_entries
        self.entries = {}

    def lookup(self, address):
        if address in self.entries:
            return {'physical_address': self.entries[address]}
        else:
            return None

    def update(self, address, physical_address):
        self.entries[address] = physical_address


class DataCache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def update_cache(self, address, is_write):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return  # Hit
        else:
            cache_set[line_offset] = address
            return  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset


class L2Cache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def update_cache(self, address, is_write):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return  # Hit
        else:
            cache_set[line_offset] = address
            return  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset


class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets',I apologize for the incomplete response. Here's the rest of the code:

```python
'0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)

    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            if ':' in line:
                address, access_type = line.strip().split(':')
                self.trace_data.append((int(address.strip(), 16), access_type.strip()))

    def simulate(self):
        for address, access_type in self.trace_data:
            # Check DTLB
            dtlb_lookup_result = self.dtlb.lookup(address)
            if dtlb_lookup_result is None:
                self.misses['dtlb'] += 1
                dtlb_lookup_result = self.pt.lookup(address)
                if dtlb_lookup_result is None:
                    self.misses['pt'] += 1
                    physical_address = address
                else:
                    self.hits['pt'] += 1
                    physical_address = dtlb_lookup_result['physical_address']
                    self.dtlb.update(address, physical_address)
            else:
                self.hits['dtlb'] += 1
                physical_address = dtlb_lookup_result['physical_address']

            # Access DC
            self.dc.update_cache(physical_address, access_type == 'W')

            # Access L2 cache
            self.l2.update_cache(physical_address, access_type == 'W')

        print('DTLB hits:', self.hits['dtlb'])
        print('DTLB misses:', self.misses['dtlb'])
        print('PT hits:', self.hits['pt'])
        print('PT misses:', self.misses['pt'])
        print('DC hits:', self.hits['dc'])
        print('DC misses:', self.misses['dc'])
        print('L2 cache hits:', self.hits['l2'])
        print('L2 cache misses:', self.misses['l2'])


SyntaxError: ignored

In [51]:
class DTLB:
    def __init__(self, num_sets, set_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def lookup(self, address):
        set_index, tag = self.get_indices(address)
        cache_set = self.cache[set_index]
        if tag in cache_set:
            return {'physical_address': address}
        else:
            return None

    def update(self, address, physical_address):
        set_index, tag = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_set.remove(None)
        cache_set.append(tag)

    def get_indices(self, address):
        set_index = (address // self.set_size) % self.num_sets
        tag = address // self.set_size
        return set_index, tag


class PageTable:
    def __init__(self, num_entries):
        self.num_entries = num_entries
        self.entries = {}

    def lookup(self, address):
        if address in self.entries:
            return {'physical_address': self.entries[address]}
        else:
            return None

    def update(self, address, physical_address):
        self.entries[address] = physical_address


class DataCache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def update_cache(self, address, is_write):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return  # Hit
        else:
            cache_set[line_offset] = address
            return  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset


class L2Cache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def update_cache(self, address, is_write):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return  # Hit
        else:
            cache_set[line_offset] = address
            return  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset


class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets',I apologize for the incomplete response. Here's the rest of the code:'0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)

    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            if ':' in line:
                address, access_type = line.strip().split(':')
                self.trace_data.append((int(address.strip(), 16), access_type.strip()))

    def simulate(self):
        for address, access_type in self.trace_data:
            # Check DTLB
            dtlb_lookup_result = self.dtlb.lookup(address)
            if dtlb_lookup_result is None:
                self.misses['dtlb'] += 1
                dtlb_lookup_result = self.pt.lookup(address)
                if dtlb_lookup_result is None:
                    self.misses['pt'] += 1
                    physical_address = address
                else:
                    self.hits['pt'] += 1
                    physical_address = dtlb_lookup_result['physical_address']
                    self.dtlb.update(address, physical_address)
            else:
                self.hits['dtlb'] += 1
                physical_address = dtlb_lookup_result['physical_address']

            # Access DC
            self.dc.update_cache(physical_address, access_type == 'W')

            # Access L2 cache
            self.l2.update_cache(physical_address, access_type == 'W')

print('DTLB hits:', simulator.hits['dtlb'])
print('DTLB misses:', simulator.misses['dtlb'])
print('PT hits:', simulator.hits['pt'])
print('PT misses:', simulator.misses['pt'])
print('DC hits:', simulator.hits['dc'])
print('DC misses:', simulator.misses['dc'])
print('L2 cache hits:', simulator.hits['l2'])
print('L2 cache misses:', simulator.misses['l2'])

SyntaxError: ignored

Test2

In [53]:
class DTLB:
    def __init__(self, num_sets, set_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    # Rest of the class code...

class PageTable:
    def __init__(self, num_entries):
        self.num_entries = num_entries
        self.entries = {}

    # Rest of the class code...

class DataCache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    # Rest of the class code...

class L2Cache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    # Rest of the class code...

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    # Rest of the class code...

simulator = MemoryHierarchySimulator(config_file_path, trace_file_path)

# Read the configuration file
simulator.read_configuration()

# Initialize the memory hierarchy components
simulator.initialize_memory_hierarchy()

# Read the trace data from the uploaded trace.dat file
simulator.read_trace_data()

# Simulate the memory hierarchy
simulator.simulate()

# Print the simulation results
print('DTLB hits:', simulator.hits['dtlb'])
print('DTLB misses:', simulator.misses['dtlb'])
print('PT hits:', simulator.hits['pt'])
print('PT misses:', simulator.misses['pt'])
print('DC hits:', simulator.hits['dc'])
print('DC misses:', simulator.misses['dc'])
print('L2 cache hits:', simulator.hits['l2'])
print('L2 cache misses:', simulator.misses['l2'])


AttributeError: ignored

Test

In [47]:
class DTLB:
    def __init__(self, num_sets, set_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def lookup(self, address):
        set_index, tag = self.get_indices(address)
        cache_set = self.cache[set_index]
        if tag in cache_set:
            return {'physical_address': address}
        else:
            return None

    def update(self, address, physical_address):
        set_index, tag = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_set.remove(None)
        cache_set.append(tag)

    def get_indices(self, address):
        set_index = (address // self.set_size) % self.num_sets
        tag = address // self.set_size
        return set_index, tag


class PageTable:
    def __init__(self, num_entries):
        self.num_entries = num_entries
        self.entries = {}

    def lookup(self, address):
        if address in self.entries:
            return {'physical_address': self.entries[address]}
        else:
            return None

    def update(self, address, physical_address):
        self.entries[address] = physical_address


class DataCache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def update_cache(self, address, is_write):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return  # Hit
        else:
            cache_set[line_offset] = address
            return  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset


class L2Cache:
    def __init__(self, num_sets, set_size, line_size):
        self.num_sets = num_sets
        self.set_size = set_size
        self.line_size = line_size
        self.cache = [[None for _ in range(set_size)] for _ in range(num_sets)]

    def update_cache(self, address, is_write):
        set_index, line_offset = self.get_indices(address)
        cache_set = self.cache[set_index]
        cache_line = cache_set[line_offset]
        if cache_line is not None and cache_line == address:
            return  # Hit
        else:
            cache_set[line_offset] = address
            return  # Miss

    def get_indices(self, address):
        set_index = (address // self.line_size) % self.num_sets
        line_offset = (address // self.line_size) % self.set_size
        return set_index, line_offset


class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)

    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            if ':' in line:
                address, access_type = line.strip().split(':')
                self.trace_data.append((int(address.strip(), 16), access_type.strip()))

    def simulate(self):
        for address, access_type in self.trace_data:
            # Check DTLB
            dtlb_lookup_result = self.dtlb.lookup(address)
            if dtlb_lookup_result is None:
                self.misses['dtlb'] += 1
                dtlb_lookup_result = self.pt.lookup(address)
                if dtlb_lookup_result is None:
                    self.misses['pt'] += 1
                    physical_address = address
                else:
                    self.hits['pt'] += 1
                    physical_address = dtlb_lookup_result['physical_address']
                    self.dtlb.update(address, physical_address)
            else:
                self.hits['dtlb'] += 1
                physical_address = dtlb_lookup_result['physical_address']

            # Access DC
            self.dc.update_cache(physical_address, access_type == 'W')

            # Access L2 cache
            self.l2.update_cache(physical_address, access_type == 'W')

print('DTLB hits:', simulator.hits['dtlb'])
print('DTLB misses:', simulator.misses['dtlb'])
print('PT hits:', simulator.hits['pt'])
print('PT misses:', simulator.misses['pt'])
print('DC hits:', simulator.hits['dc'])
print('DC misses:', simulator.misses['dc'])
print('L2 cache hits:', simulator.hits['l2'])
print('L2 cache misses:', simulator.misses['l2'])


DTLB hits: 0
DTLB misses: 0
PT hits: 0
PT misses: 0
DC hits: 0
DC misses: 0
L2 cache hits: 0
L2 cache misses: 0


end

In [ ]:
from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

    # ... (existing code)


    def print_configuration(self):
    #def print_configuration(self):
        print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")

from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                 #    config_file = files.upload()
                      config_file_path = list(config_file.keys())[0]
                      self.config_file_path = config_file_path

                 # Upload trace.dat file
    trace_file = files.upload()
    trace_file_path = list(trace_file.keys())[0]
    self.trace_file_path = trace_file_path

    # Read the configuration from the uploaded trace.config file
    self.read_configuration()

    # Initialize the memory hierarchy components
    self.initialize_memory_hierarchy()

    # Read the trace data from the uploaded trace.dat file
    self.read_trace_data()
                  #  self.uploaded = files.upload()
                 # lTo use the code in Google Colab, you need to upload the `trace.config` and `trace.dat` files to the Colab runtime environment. You can do this using the file upload feature in Colab. Here's an updated version of the code that includes the file upload steps:


from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

    # ... (existing code)


    def print_configuration(self):
        print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")


    def print_reference_information(self):
          #def print_simulation_statistics(self):
        print("Simulation Statistics")

        # Print the number of hits, misses, and hit ratio for each level of the hierarchy
        print("DTLB:")
        print(f"DTLB hits: {self.hits['dtlb']}")
        print(f"DTLB misses: {self.misses['dtlb']}")
        dtlb_hit_ratio = self.hits['dtlb'] / (self.hits['dtlb'] + self.misses['dtlb'])
        print(f"DTLB hit ratio: {dtlb_hit_ratio:.6f}")

        print("Page Table:")
        print(f"PT hits: {self.hits['pt']}")
        print(f"PT faults: {self.misses['pt']}")
        pt_hit_ratio = self.hits['pt'] / (self.hits['pt'] + self.misses['pt'])
        print(f"PT hit ratio: {pt_hit_ratio:.6f}")

        if self.config_data.get('DC', 'n') == 'y':
            print("Data Cache:")
            print(f"DC hits: {self.hits['dc']}")
            print(f"DC misses: {self.misses['dc']}")
            dc_hit_ratio = self.hits['dc'] / (self.hits['dc'] + self.misses['dc'])
            print(f"DC hit ratio: {dc_hit_ratio:.6f}")

        if self.config_data.get('L2 cache', 'n') == 'y':
            print("L2 Cache:")
            print(f"L2 hits: {self.hits['l2']}")
            print(f"L2 misses: {self.misses['l2']}")
            l2_hit_ratio = self.hits['l2'] / (self.hits['l2'] + self.misses['l2'])
            print(f"L2 hit ratio: {l2_hit_ratio:.6f}")

        # Print the number of reads, writes, read ratio, total memory references, accesses to the page table, and disk references
        total_reads = self.accesses['R']
        total_writes = self.accesses['W']
        total_memory_references = total_reads + total_writes
        read_ratio = total_reads / total_memory_references

        print(f"Total reads: {total_reads}")
        print(f"Total writes: {total_writes}")
        print(f"Read ratio: {read_ratio:.6f}")
        print(f"Total memory references: {total_memory_references}")
        print(f"Accesses to the page table: {self.page_table_accesses}")
        print(f"Disk references: {self.disk_references}")

from google.colab import files

class MemoryHierarchySimulator:
    def __init__(self, config_file_path, trace_file_path):
        self.config_file_path = config_file_path
        self.trace_file_path = trace_file_path
        self.config_data = {}
        self.trace_data = []
        self.hits = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.misses = {'dtlb': 0, 'pt': 0, 'dc': 0, 'l2': 0}
        self.dtlb = None
        self.pt = None
        self.dc = None
        self.l2 = None

    def read_configuration(self):
        with open(self.config_file_path, 'r') as config_file:
            config_lines = config_file.readlines()

        for line in config_lines:
            if ':' in line:
                key, value = line.strip().split(':')
                self.config_data[key.strip()] = value.strip()

    def initialize_memory_hierarchy(self):
        # Initialize the memory hierarchy components based on the configuration
        num_dtlb_sets = int(self.config_data.get('Data TLB configuration.Number of sets', '0'))
        dtlb_set_size = int(self.config_data.get('Data TLB configuration.Set size', '0'))
        num_pt_entries = int(self.config_data.get('Page Table configuration.Number of virtual pages', '0'))
        num_dc_sets = int(self.config_data.get('Data Cache configuration.Number of sets', '0'))
        dc_set_size = int(self.config_data.get('Data Cache configuration.Set size', '0'))
        dc_line_size = int(self.config_data.get('Data Cache configuration.Line size', '0'))
        num_l2_sets = int(self.config_data.get('L2 Cache configuration.Number of sets', '0'))
        l2_set_size = int(self.config_data.get('L2 Cache configuration.Set size', '0'))
        l2_line_size = int(self.config_data.get('L2 Cache configuration.Line size', '0'))

        # Initialize DTLB
        self.dtlb = DTLB(num_dtlb_sets, dtlb_set_size)

        # Initialize PT
        self.pt = PageTable(num_pt_entries)

        # Initialize DC
        self.dc = DataCache(num_dc_sets, dc_set_size, dc_line_size)

        # Initialize L2 cache
        self.l2 = L2Cache(num_l2_sets, l2_set_size, l2_line_size)


    def read_trace_data(self):
        with open(self.trace_file_path, 'r') as trace_file:
            trace_lines = trace_file.readlines()

        for line in trace_lines:
            self.trace_data.append(line.strip())


    # ... (existing code)

    def simulate_memory_hierarchy(self):
        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Update the TLB, PT, and associated caches accordingly
                if self.config_data.get('TLB', 'n') == 'y':
                    tlb_result = self.dtlb.lookup(virtual_address)
                    if tlb_result is None:
                        tlb_result = self.pt.lookup(virtual_address)
                        if tlb_result is not None:
                            self.dtlb.update(virtual_address, tlb_result['physical_address'])

                if self.config_data.get('PT', 'n') == 'y':
                    pt_result = self.pt.lookup(virtual_address)
                    if pt_result is None:
                        # Handle page fault
                        pass

                # Perform physical address translation
                physical_address = self.pt.translate(virtual_address)

            if self.config_data.get('DC', 'n') == 'y':
                # Simulate the behavior of the DC
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.dc.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.dc.update_cache(physical_address, access_type == 'W', write_allocate=False)

            if self.config_data.get('L2 cache', 'n') == 'y':
                # Simulate the behavior of the L2 cache
                if self.config_data.get('Write through/no write allocate', 'n') == 'n':
                    # Perform write-allocate and write-back policy
                    self.l2.update_cache(physical_address, access_type == 'W')
                else:
                    # Perform no write-allocate and write-through policy
                    self.l2.update_cache(physical_address, access_type == 'W', write_allocate=False)

    # ... (existing code)


    def print_configuration(self):
        print("Configuration of the Memory Hierarchy")
        print("Data TLB configuration:")
        print(f"Number of sets: {self.dtlb.num_sets}")
        print(f"Set size: {self.dtlb.set_size}")

        print("Page Table configuration:")
        print(f"Number of virtual pages: {self.pt.num_entries}")

        print("Data Cache configuration:")
        print(f"Number of sets: {self.dc.num_sets}")
        print(f"Set size: {self.dc.set_size}")
        print(f"Line size: {self.dc.line_size}")
        print(f"Write through/no write allocate: {'y' if self.dc.write_allocate else 'n'}")

        print("L2 Cache configuration:")
        print(f"Number of sets: {self.l2.num_sets}")
        print(f"Set size: {self.l2.set_size}")
        print(f"Line size: {self.l2.line_size}")
        print(f"Write through/no write allocate: {'y' if self.l2.write_allocate else 'n'}")


    def print_reference_information(self):
        print("Information about each Reference")
        print("Virtual Virt. Page TLB TLB TLB PT Phys DC DC L2 L2")
        print("Address Page # Off Tag Ind Res. Res. Pg # DC Tag Ind Res. L2 Tag Ind Res.")
        print("-------- ------ ---- ------ --- ---- ---- ---- ------ --- ---- ------ --- ----")

        for reference in self.trace_data:
            # Extract the access type and address from the reference
            access_type, address = reference.split(':')

            if self.config_data.get('Virtual addresses', 'n') == 'y':
                # Perform virtual-to-physical address translation
                virtual_address = int(address, 16)

                # Obtain the relevant information
                virtual_page_number = virtual_address >> self.pt.page_offset_bits
                page_offset = virtual_address & ((1 << self.pt.page_offset_bits) - 1)
                tlb_tag, tlb_index, tlb_result = self.dtlb.lookup_info(virtual_page_number)
                pt_result = self.pt.lookup_info(virtual_page_number)


Saving trace.config to trace (1).config


NameError: ignored